In [ ]:
#모듈화를 위한 코드
%%writefile ./test.py
def test() :
    print("test")

In [1]:
import pymysql as ps
import pandas as pd
from datetime import datetime, timedelta
import time
from sqlalchemy import create_engine

def con():
    conn = ps.connect(host='localhost', user='root', passwd='12345', db='aqu4men')
    return conn

def insert_db(table, calumns, values):
    conn = con()
    curs = conn.cursor()
    sql = f"insert into {table} {calumns} values {values};"
    curs.execute(sql)
    conn.commit()
    curs.close()
    conn.close()
    return sql

def select_db(calumn,table, where):
    conn = con()
    curs = conn.cursor()
    sql = f"selet {calumn} from {table} where {where} "
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    return sql

In [ ]:
import os
from flask import Flask, request, redirect, jsonify
from flask import Flask, send_from_directory
from flask_cors import CORS, cross_origin
from threading import Lock

#전역변수
current_object=0
current_take=0
coasters=0
cupholder=0
led = 2


#리액트 플라스크 연동 위한 코드
async_mode = None
app = Flask(__name__, static_folder='build') # Change your react-app name 'my-react-app'
app.config['SECRET_KEY'] = 'secret!'
thread_lock = Lock()
cors = CORS(app)
app.config["CORS_HEADERS"] = 'Content-Type'

@app.route('/', defaults={'path': ''})

@app.route('/<path:path>')
def serve(path):
    if path != "" and os.path.exists(app.static_folder + '/' + path):
        return send_from_directory(app.static_folder, path)
    else:
        return send_from_directory(app.static_folder, 'index.html')

    

###########################################################회원가입###########################################################
@app.route('/join')
def join():
    params = requenst.get_json()
    userid = params['email']
    userpw = params['password']
    username = params['name']
    userbirthyear = params['birthyear']
    userphone = params['phone']
    table = 't_user'
    calumns= '(user_id, user_pw, user_name, user_weight, user_birthyear, user_phone)'
    values = f'("{userid}", "{userpw}", "{username}", {int(userweight)}, "{userbirthyear}", "{userphone}")'
    sql=insert_db(table, calumns, values)
    conn = con()
    curs = conn.cursor()
    sql = f'select user_no from t_user where user_id = "{userid}"'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    id = result[0][0]
    table3 = 't_achievement'
    calumns3= '(a_time, user_no, 1st_achievement)'
    values3 = f'(now(), "{id}", "F")'
    sq3=insert_db(table3, calumns3, values3) 
    return '{"return":"회원가입완료"!}'
    
    
############################################################로그인############################################################
@app.route('/login')
def login():
    params = requenst.get_json()
    userid = params['email']
    userpw = params['password']
    conn = con()
    curs = conn.cursor()
    sql = f'select user_pw from t_user where user_id = "{userid}"'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    if userpw == (result[0][0]):
        return "{'return':'로그인 승인'}"
    else:
        return "{'return':'로그인 실패'}"



#########################################################메인 페이지#########################################################
@app.route('/main', methods=['GET','POST'])
def intake():
#목표 섭취량 계산 공식    
    conn = con()
    curs = conn.cursor()
    sql = f'select user_weight from t_user where {int(id)} = user_no'
    curs.execute(sql)
    result = curs.fetchall()  #튜플형태로 반환
    curs.close()
    conn.close()
    current_object = result[0][0]*33
    return (f"\{'current_object':'{curren_object}'\}")

#현재섭취량(1) 계산 공식 (이전 무게 - 현재 무게)
    conn = con()
    curs = conn.cursor()
    sql = f'select weight, wv_time from t_coasters where {int(id)} = user_no order by wv_time desc limit 2'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    coasters=result[1][0]-result[0][0]
    if coasters >=50 :
        coasters = coasters
    else: 
        coasters = 0

#현재 섭취량(2) 계산 공식 
    conn = con()
    curs = conn.cursor()
    sql = f'select x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20,\
x21,y21,x22,y22,x23,y23,x24,y24,x25,y25,x26,y26,x27,y27,x28,y28,x29,y29,x30,y30,t from t_cupholder where {int(id)} = user_no order by av_time desc limit 1;'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    df = pd.DataFrame(result, columns=['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5',
                                       'x6','y6','x7','y7','x8','y8','x9','y9','x10','y10',
                                       'x11','y11','x12','y12','x13','y13','x14','y14','x15','y15',
                                       'x16','y16','x17','y17','x18','y18','x19','y19','x20','y20',
                                       'x21','y21','x22','y22','x23','y23','x24','y24','x25','y25',
                                       'x26','y26','x27','y27','x28','y28','x29','y29','x30','y30','t'])
    cupholder = '머신러닝.fit()'
    sql2 = f'insert into t_cupholder (label) values ({cupholder})'
    current_take = int(coasters) + int(cupholder)
    return f"[\{current_object:{current_object}\},\{current_take:{current_take}\}]"



#########################################################차트 페이지#########################################################
@app.route('/dashboard')
def dashboard():           
    return '''
        [{"time":"08:00", "mean":"500", "take":"300"},
         {"time":"10:00", "mean":"600", "take":"350"},
         {"time":"12:00", "mean":"750", "take":"500"},
         {"time":"14:00", "mean":"900", "take":"600"},
         {"time":"16:00", "mean":"1000", "take":"650"},
         {"time":"18:00", "mean":"1200", "take":"800"},
         {"time":"20:00", "mean":"1400", "take":"1000"}]
         '''
#시간대별로 섭취량 보내주는 쿼리문

############################################################알림############################################################
@app.route('/alret/<int:id>')
def alret(id):
#알림 계산 공식 (현재시각 - [무게or각도변화시각])
    conn = con()
    curs = conn.cursor()
    sql = f'select wv_time from t_coasters where {int(id)} = user_no order by wv_time desc limit 1'
    curs.execute(sql)
    result = curs.fetchall()
    sql2 = f'select av_time from t_cupholder where {int(id)} = user_no order by av_time desc limit 1'
    curs.execute(sql2)
    result2 = curs.fetchall()
    if result>result2 :
        result=result
    else:
        result=result2
    sql3 = f'select current_timestamp'
    curs.execute(sql3)
    current_time = curs.fetchall()
    curs.close()
    conn.close()
    a_time = current_time[0][0]-result[0][0] #a_time은 datetime.timedelta 타입이다. 
                                             #타임델타는 두 날짜 또는 사간의 차이가 어느정도 인지 나타낼때 사용하는 모듈. (+,-사용가능)

    #timedelta to int 함수
    def timedelta2int(td):
        res = round(td.microseconds/float(1000000)) + (td.seconds+td.days*24*3600)
        return res

    a_time = timedelta2int(a_time)
    print(a_time)
    if a_time > 7200:
        global led
        led = 1
        return 'blink on'
    else:
        led = 0
        return 'led off '


############################################################달성############################################################
@app.route('/achivement')
def achivement(id):
    id=1
#목표 섭취량 계산 공식    
    conn = con()
    curs = conn.cursor()
    sql = f'select user_weight from t_user where {int(id)} = user_no'
    curs.execute(sql)
    result = curs.fetchall()  #튜플형태로 반환
    curs.close()
    conn.close()
    current_object = result[0][0]*33
    return (f"\{'current_object':'{curren_object}'\}")

#현재섭취량(1) 계산 공식 (이전 무게 - 현재 무게)
    conn = con()
    curs = conn.cursor()
    sql = f'select weight, wv_time from t_coasters where {int(id)} = user_no order by wv_time desc limit 2'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    coasters=result[1][0]-result[0][0]
    if coasters >=50 :
        coasters = coasters
    else: 
        coasters = 0

#현재 섭취량(2) 계산 공식 
    conn = con()
    curs = conn.cursor()
    sql = f'select x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20,\
x21,y21,x22,y22,x23,y23,x24,y24,x25,y25,x26,y26,x27,y27,x28,y28,x29,y29,x30,y30,t from t_cupholder where {id} = user_no order by av_time desc limit 1;'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    df = pd.DataFrame(result, columns=['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5',
                                       'x6','y6','x7','y7','x8','y8','x9','y9','x10','y10',
                                       'x11','y11','x12','y12','x13','y13','x14','y14','x15','y15',
                                       'x16','y16','x17','y17','x18','y18','x19','y19','x20','y20',
                                       'x21','y21','x22','y22','x23','y23','x24','y24','x25','y25',
                                       'x26','y26','x27','y27','x28','y28','x29','y29','x30','y30','t'])
    cupholder = 0 #머신러닝.fit()
    sql2 = f'insert into t_cupholder (label) values ({cupholder})'
    current_take = int(coasters) + int(cupholder)
    return f"[\{current_object:{current_object}\},\{current_take:{current_take}\}]"

# 조건문으로 보상획득 시간이 오늘이 아닌 경우에만 이 함수 시작하도록 설정
# if :

    #첫째 날
    if (current_object - current_take) <= 0:
        achiv = '1st'
        conn = con()
        curs = conn.cursor()
        id=1
        sql = f'select 1st_achievement from t_achievement where user_no = {int(id)} order by a_time desc limit 1'
        curs.execute(sql)
        result = curs.fetchall()
        curs.close()
        conn.close()
        result = result[0][0]
        return "1일차 달성"
    #둘째 날    
        if (result == 'T'):
            achiv = '2nd'
            conn = con()
            curs = conn.cursor()
            sql = f'select 2nd_achievement from t_achievement where user_no = {int(id)} order by a_time desc limit 1'
            curs.execute(sql)
            result = curs.fetchall()
            curs.close()
            conn.close()
            result = result[0][0]
            return "2일차 달성"

    #셋째 날
            if (result == 'T'):
                achive = '3rd'
                table = "t_achievement"
                calumns = '(a_time, user_no, 1st_achievement, 2nd_achievement, reward)'
                values = f"(now(), {id}, 'F', 'F', 'T')"
                insert_db(table, calumns, values) 
                # nft_index 넣어주는 쿼리문
                conn = con()
                curs = conn.cursor()
                sql = f'update t_achievement SET nft_index = (SELECT min(nft_index) FROM t_reward WHERE user_no IS NULL) WHERE user_no = f"{int(id)}" AND reward = "T" AND nft_index IS NULL'
                curs.execute(sql)
                result = curs.fetchall()
                curs.close()
                conn.close()
                return "JSON 코드"
            else:
                table = "t_achievement"
                calumns = '(a_time, user_no, 1st_achievement, 2nd_achievement)'
                values = f"(now(), {int(id)}, 'T', 'T')"
                insert_db(table, calumns, values)
                return "3일차 미달성"
        else:
            print(result)
            table = "t_achievement"
            calumns = '(a_time, user_no, 1st_achievement)'
            values = f"(now(), {id}, 'T')"
            insert_db(table, calumns, values)
            return "2일차 미달성"
    else: 
        return "1일차 미달성"
else:
    return "이미 보상을 획득했습니다!"
 # 조건문으로 보상획득 시간이 오늘이 아닌 경우에만 이 함수 시작하도록 설정


############################################################보상############################################################
@app.route('/reward/<int:id>')
def reward(id):
    conn = con()
    curs = conn.cursor()
    sql = f'select nft from t_reward where {int(id)} = user_no'
    curs.execute(sql)
    result = curs.fetchall()
    curs.close()
    conn.close()
    result = result[0][0]

    
    
########################################################컬렉션 페이지########################################################
# @app.route('/collection', methods=['GET', 'POST'])
# def collection():
#     return f'collection {}'
# Q.JSON 코드 업로드방법?
    
    
    
#########################################################설정 페이지#########################################################
@app.route('/setting/<int:id>', methods=['GET', 'POST'])
def setting(id):
    params = requenst.get_json()
    userpw = params['password']
    userphone = params['phone']
    user_led_alarm = params['led']
    user_message_alarm = params['message']
    return 'setting'
# Q. DB에 설정 값 저장하는 공간 필요할 듯    

    
###########################################################로드셀###########################################################
@app.route('/insert/loadcell', methods=['GET'])
def insert_loadcell():
    read = "can not read"
    if request.method == 'GET':
        request.get_json()
        read = request.args['loadcell']
        print('sensing data : ',read)
        
        conn = con()
        curs = conn.cursor()
        sql = f'select weight, wv_time from t_coasters where {id} = user_no order by wv_time desc limit 1'
        curs.execute(sql)
        result = curs.fetchall()
        coasters=result[0][0]
        
        if abs(int(coasters) - int(read))>30:
            table = 't_coasters'
            print('table :',table)
            calumns= '(coaster_seq, user_no, wv_time, weight)'
            print('calumns :',calumns)
            values = f'("productID", "1", now(), {int(read)})'
            print('values :',values)
            sql=insert_db(table, calumns, values)
            print(sql)
            return "value of loadcell is %s" %read 
    else:
        return read
    
###########################################################가속도###########################################################
@app.route('/insert/mpu', methods=['GET'])
def insert_mpu():
    read = "can not read"
    if request.method == 'GET':
        request.get_json()
        x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10 = request.args['x1'],request.args['y1'],request.args['x2'],\
request.args['y2'],request.args['x3'],request.args['y3'],request.args['x4'],request.args['y4'],request.args['x5'],\
request.args['y5'],request.args['x6'],request.args['y6'],request.args['x7'],request.args['y7'],request.args['x8'],\
request.args['y8'],request.args['x9'],request.args['y9'],request.args['x10'],request.args['y10']
        x11,y11,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20 = request.args['x11'],request.args['y11'],\
request.args['x12'],request.args['y12'],request.args['x13'],request.args['y13'],request.args['x14'],request.args['y14'],\
request.args['x15'],request.args['y15'],request.args['x16'],request.args['y16'],request.args['x17'],request.args['y17'],\
request.args['x18'],request.args['y18'],request.args['x19'],request.args['y19'],request.args['x20'],request.args['y20']
        x21,y21,x22,y22,x23,y23,x24,y24,x25,y25,x26,y26,x27,y27,x28,y28,x29,y29,x30,y30 = request.args['x21'],request.args['y21'],\
request.args['x22'],request.args['y22'],request.args['x23'],request.args['y23'],request.args['x24'],request.args['y24'],\
request.args['x25'],request.args['y25'],request.args['x26'],request.args['y26'],request.args['x27'],request.args['y27'],\
request.args['x28'],request.args['y28'],request.args['x29'],request.args['y29'],request.args['x30'],request.args['y30']
#         T,Label = request.args['T'],request.args['Label']
        t=3
        label=150
        table = 't_cupholder'
        print('table :',table)
        calumns= '(product_id, user_no, av_time, x1, y1, x2, y2, x3, y3, x4, y4, x5, y5, x6, y6, x7, y7, x8, y8, x9, y9, x10, y10, X11,\
y11, x12, y12, x13, y13, x14, y14, x15, y15, x16, y16, x17, y17, x18, y18, x19, y19, x20, y20, x21, y21, x22, y22, x23, y23,\
x24, y24, x25, y25, x26, y26, x27, y27, x28, y28, x29, y29, x30, y30, t, label)'
        print('calumns :',calumns)
        values = f'("productID", "1", now(), {int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}, {int(x3)}, {int(y3)}, {int(x4)}, {int(y4)},\
{int(x5)}, {int(y5)}, {int(x6)}, {int(y6)}, {int(x7)}, {int(y7)}, {int(x8)}, {int(y8)}, {int(x9)}, {int(y9)}, {int(x10)},\
{int(y10)}, {int(x11)}, {int(y11)}, {int(x12)}, {int(y12)}, {int(x13)}, {int(y13)}, {int(x14)}, {int(y14)}, {int(x15)},\
{int(y15)}, {int(x16)}, {int(y16)}, {int(x17)}, {int(y17)}, {int(x18)}, {int(y18)}, {int(x19)}, {int(y19)}, {int(x20)},\
{int(y20)}, {int(x21)}, {int(y21)}, {int(x22)}, {int(y22)}, {int(x23)}, {int(y23)}, {int(x24)}, {int(y24)}, {int(x25)},\
{int(y25)}, {int(x26)}, {int(y26)}, {int(x27)}, {int(y27)}, {int(x28)}, {int(y28)}, {int(x29)}, {int(y29)}, {int(x30)},\
{int(y30)}, {t}, {label})'
        print('values :',values)
        sql=insert_db(table, calumns, values)
        print(sql)
        return "value of MPU6050 is done"
    else:
        return read

    
if __name__ == '__main__':
    app.run(host='localhost', port=9500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9500/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Aug/2022 10:45:31] "GET /videos/water.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [26/Aug/2022 10:46:25] "GET /videos/water.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [26/Aug/2022 14:17:51] "GET /videos/water.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [26/Aug/2022 14:39:16] "GET /videos/water.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [26/Aug/2022 14:39:22] "GET /videos/water.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [26/Aug/2022 14:39:48] "GET /Set HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 14:39:58] "GET /Set HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 14:39:59] "GET /Set HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 21:02:07] "GET /Set HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 21:03:01] "GET /Chart HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 21:03:01] "GET /Chart HTTP/1.1" 304 -
127.0.0.1 - - [26/Aug/2022 21:03:02] "GET /Chart HTTP/1.1" 304 -


http://192.168.70.170:5000/insert/mpu?x1=200&y1=100&x2=200&y2=100&x3=200&y3=100&x4=200&y4=100&x5=200&y5=100&x6=200&y6=100&x7=200&y7=100&x8=200&y8=100&x9=200&y9=100&x10=200&y10=100&x11=200&y11=100&x12=200&y12=100&x13=200&y13=100&x14=200&y14=100&x15=200&y15=100&x16=200&y16=100&x17=200&y17=100&x18=200&y18=100&x19=200&y19=100&x20=200&y20=100&x21=200&y21=100&x22=200&y22=100&x23=200&y23=100&x24=200&y24=100&x25=200&y25=100&x26=200&y26=100&x27=200&y27=100&x28=200&y28=100&x29=200&y29=100&x30=200&y30=100